In [1]:
import yaml

from primaite.game.game import PrimaiteGame
from primaite.session.environment import PrimaiteGymEnv
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.system.applications.database_client import DatabaseClient, DatabaseClientConnection
from primaite.simulator.system.applications.red_applications.data_manipulation_bot import DataManipulationBot
from primaite.simulator.system.services.database.database_service import DatabaseService
from primaite import getLogger, PRIMAITE_PATHS

In [2]:
with open(PRIMAITE_PATHS.user_config_path / "example_config" / "data_manipulation.yaml") as f:
        cfg = yaml.safe_load(f)
game = PrimaiteGame.from_config(cfg)
uc2_network = game.simulation.network

In [3]:
client_1: Computer = uc2_network.get_node_by_hostname("client_1")
db_manipulation_bot: DataManipulationBot = client_1.software_manager.software.get("DataManipulationBot")

database_server: Server = uc2_network.get_node_by_hostname("database_server")
db_service: DatabaseService = database_server.software_manager.software.get("DatabaseService")

web_server: Server = uc2_network.get_node_by_hostname("web_server")
db_client: DatabaseClient = web_server.software_manager.software.get("DatabaseClient")
db_connection: DatabaseClientConnection = db_client.get_new_connection()
db_service.backup_database()

# First check that the DB client on the web_server can successfully query the users table on the database
assert db_connection.query("SELECT")

db_manipulation_bot.data_manipulation_p_of_success = 1.0
db_manipulation_bot.port_scan_p_of_success = 1.0

# Now we run the DataManipulationBot
db_manipulation_bot.attack()

# Now check that the DB client on the web_server cannot query the users table on the database
assert not db_connection.query("SELECT")

# Now restore the database
db_service.restore_backup()

# Now check that the DB client on the web_server can successfully query the users table on the database
assert db_connection.query("SELECT")

In [4]:
router = uc2_network.get_node_by_hostname("router_1")

In [5]:
router.acl.show()

+-----------------------------------------------------------------------------------------------------------------+
|                                           router_1 Access Control List                                          |
+-------+--------+----------+--------+--------------+-------------+--------+--------------+-------------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port    | Dst IP | Dst Wildcard | Dst Port    | Matched |
+-------+--------+----------+--------+--------------+-------------+--------+--------------+-------------+---------+
| 18    | PERMIT | ANY      | ANY    | ANY          | 5432 (5432) | ANY    | ANY          | 5432 (5432) | 4       |
| 19    | PERMIT | ANY      | ANY    | ANY          | 53 (53)     | ANY    | ANY          | 53 (53)     | 0       |
| 20    | PERMIT | ANY      | ANY    | ANY          | 21 (21)     | ANY    | ANY          | 21 (21)     | 0       |
| 21    | PERMIT | ANY      | ANY    | ANY          | 80 (80)     | ANY 

In [6]:
router.acl.is_permitted()

TypeError: AccessControlList.is_permitted() missing 1 required positional argument: 'frame'